<a href="https://colab.research.google.com/github/farhanhs/machine-learning-project/blob/master/house_linear_myfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.stats import skew
from scipy.stats import boxcox_normmax
from scipy.special import boxcox1p
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
train_data = pd.read_csv('train.csv',encoding='utf-8')
test_data = pd.read_csv('test.csv',encoding='utf-8')


In [ ]:
data = pd.concat([ train_data, test_data], axis = 0)

In [ ]:
y = train_data["SalePrice"]

In [ ]:
#測試畫出價格散佈圖後的偏度
%matplotlib inline
print('skew:',skew(y))
print('lambda',boxcox_normmax(1+y))
sns.distplot(y)

In [ ]:
#對y取log作為新的答案
%matplotlib inline
y_train = np.log1p(y)
print('skew:',skew(y_train))
sns.distplot(y_train)

In [ ]:
data['Alley'].value_counts()

In [ ]:
s = data.isna().sum()
s[s>0].sort_values(ascending=False)

In [ ]:
data = data.drop(['Id',"PoolQC","MiscFeature","Alley","Fence"],axis = 1)

In [ ]:
data.dtypes.value_counts()

In [ ]:
idx = data.dtypes != 'object'
number_idx = data.dtypes[idx].index.drop(["MSSubClass"])
number_idx

In [ ]:
data = pd.get_dummies(data)
data = pd.get_dummies(data, columns=['MSSubClass'])
data

In [ ]:
def get_skew(series):
  return skew(series.dropna() +1)
skewness = data[number_idx].apply(get_skew).sort_values(ascending = False)
need_saved = skewness[skewness >1].index
print(need_saved)

In [ ]:
def save(s):
  lamda = boxcox_normmax(s.dropna()+1)
  return boxcox1p(s,lamda)
data[need_saved] = data[need_saved].apply(save)

In [ ]:
data = data.fillna(data.median())

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
data = pd.DataFrame(scaler.fit_transform(data))
x_train = data.iloc[:len(train_data)]
x_predict = data.iloc[len(train_data):]



In [ ]:
from sklearn.linear_model import LassoCV,RidgeCV,ElasticNetCV
from sklearn.linear_model import Lasso,Ridge,ElasticNet

cv = RidgeCV(cv=10)
cv.fit(x_train, y_train)
print(cv.alpha_)
model = Ridge(cv.alpha_)
model.fit(x_train, y_train)
pre = model.predict(x_predict)
result = pd.DataFrame({
    "Id":test_data["Id"],
    "SalePrice":np.expm1(pre)
})
result.to_csv("submission_ridge.csv", encoding="utf-8", index=False)
result